
## LISTA 3 Aprendizagem de máquina (CK0193)
## Aluno: Lucas Rodrigues Aragão (538390)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.model_selection import train_test_split

def euclidian_distance(item_1, item_2, parametro_aleatorio):

  return np.linalg.norm(item_1 - item_2)

def mahalanobis_distance(item_1, item_2, covariance_matrix):
  diff =  (item_1 - item_2)
  inverse_cov_matrix = np.linalg.inv(covariance_matrix)
  mahalanobis = np.sqrt(np.dot(np.dot(diff.T, inverse_cov_matrix), diff))
  return mahalanobis

def accuracy_function(true_values: np.array,predicted: np.array):

  if len(true_values) != len(predicted):
    return 'erro de tamanho'

  num_erros = 0
  num_acertos = 0

  for i, prediction in enumerate(predicted):
    if true_values[i] == prediction:
      num_acertos +=1
    else:
      num_erros +=1
  return num_acertos/(num_acertos + num_erros)

def revocacao_function(true_values:np.array, predicted: np.array):
  num_true_positives = 0
  num_false_negatives = 0

  for i, prediction in enumerate(predicted):
    if true_values[i] == 1 and prediction == 1:
      num_true_positives +=1
    elif true_values[i] == 1 and prediction == 0:
      num_false_negatives +=1
  
  if num_true_positives+num_false_negatives == 0:
    return 0
  
  return (num_true_positives/(num_true_positives+num_false_negatives))

def precision_function(true_values: np.array, predicted: np.array):
  num_true_positives = 0
  num_false_positives = 0
  
  for i, prediction in enumerate(predicted):
    if true_values[i] == 1 and prediction == 1:
      num_true_positives +=1
    elif true_values[i] == 0 and prediction == 1:
      num_false_positives +=1
  
  if num_true_positives+num_false_positives == 0:
    return 0 
  return (num_true_positives/(num_true_positives + num_false_positives))

def f1_score_function(true_values:np.array, predicted: np.array):
  revocacao = revocacao_function(true_values, predicted)
  precision = precision_function(true_values, predicted)
  if revocacao + precision == 0:
    return 0
  return 2*((revocacao*precision)/(revocacao+precision))

def k_fold(model, model_type,features_data, label_data, num_folds = 10):


  # recebers dados de features e labels separados
  # esses dados podem estar do jeito que vieram sem alteraçoes
  #em tese o algoritmo ja recebe os dados embaralhados, por isso apenas pega as particoes
  tamanho_particoes =  int(np.ceil(len(features_data)/num_folds))
  
  #uma particao tera um tamanho menor que as demais caso os tamanhos sejam desiguais
  if tamanho_particoes*num_folds > len(features_data):
    ultima = len(features_data) - tamanho_particoes*(num_folds-1)

  f1_score = np.array([])
  acuracia = np.array([])
  revocacao = np.array([])
  precision = np.array([])

  inicio = 0
  fim = tamanho_particoes

  #em cada fold calcular as metricas de avaliacao e tirar a media
  for i in range(num_folds):
    x_teste = features_data.iloc[inicio:fim]
    y_teste = label_data[inicio:fim]

    x_train_fold  = features_data.drop(features_data.index[inicio:fim])
    y_train_fold = label_data.drop(label_data.index[inicio:fim])
    scaler = StandardScaler()
    train_fold_normalizado = pd.DataFrame(scaler.fit_transform(x_train_fold), columns= x_train_fold.columns, index = x_train_fold.index)
    x_train_fold = train_fold_normalizado.copy()
    x_teste_normalizado = pd.DataFrame(scaler.transform(x_teste), columns= x_teste.columns, index= x_teste.index)
    x_teste = x_teste_normalizado.copy()

    if model_type == 'KNN':             
      teste_predictions = model.predict_all(test_features= x_teste, training_features = x_train_fold, training_label = y_train_fold)

    if model_type == 'DT':
      model.fit(x_train_fold, y_train_fold)
      teste_predictions = model.predict(x_teste)


    y_teste = np.array(y_teste)
    acuracia = np.append(acuracia, accuracy_function(y_teste, teste_predictions))
    revocacao = np.append(revocacao, revocacao_function(y_teste, teste_predictions))
    precision = np.append(precision, precision_function(y_teste, teste_predictions))
    f1_score = np.append(f1_score, f1_score_function(y_teste, teste_predictions))

    if i+1 == num_folds-1:
      fim += ultima
    else:
      fim += tamanho_particoes
    inicio += tamanho_particoes

    
  acuracia_media = acuracia.mean()
  acuracia_desvio_padrao = np.std(acuracia)
  print(f'Valor medio da acuracia: {acuracia_media} \nDesvio padrao da acuracia: {acuracia_desvio_padrao}')

  revocacao_media = revocacao.mean()
  revocacao_desvio_padrao = np.std(revocacao)
  print(f'\nValor medio da revocacao: {revocacao_media} \nDesvio padrao da revocacao: {revocacao_desvio_padrao}')

  precision_media = precision.mean()
  precision_desvio_padrao = np.std(precision)
  print(f'\nValor medio da precisao: {precision_media} \nDesvio padrao da precisao: {precision_desvio_padrao}')

  f1_score_media = f1_score.mean()
  f1_score_desvio_padrao = np.std(f1_score)

  print(f'\nValor medio do f1_score: {f1_score_media} \nDesvio padrao do f1_score: {f1_score_desvio_padrao}')


## Criação de modelos

In [2]:

class KNN():
  def __init__(self, k = 5, distance_metric = euclidian_distance):
    if k%2 == 0:
      print('k alterado para valor impar maior')
      k +=1
    
    if distance_metric == euclidian_distance:
      self.distance_metric_name = 'Distancia Euclidiana'
    else:
      self.distance_metric_name = 'Distancia Mahalanobis'
    self.k = k
    self.distance_metric = distance_metric

  def predict_all(self, test_features, training_features, training_label:np.array ) -> np.array:
    """
    para cada instancia do teste, xte, o modelo calcula a distancia para todas as instancias de treino, xtr, e retorna a classe baseada na maioria das k vizinhas
    """
    #nao tem exatamente uma fase de treino e teste separada
    #basicamente eh so criar um vetor de distancias p saber a classe do cara,pode ser do tipo [indice, distancia] ou [classe, distancia]

    distance_metric = self.distance_metric 
    predictions = []

    covariance_matrix = np.cov(training_features.T)
    for index_1, row_1 in test_features.iterrows(): #aqui eh o laco p passar por todo mundo do teste
      row_1 = np.array(row_1)

      nearest_neighbours = [[10e6, 'neighbour label']]*self.k #vetor p guardar os k vizinhos mais proximos
      for index_2,row_2 in training_features.iterrows(): # aqui eh o laco para calcular o label do cara do laco de fora

        row_2 = np.array(row_2)
      
        if index_1 != index_2:

          distancia = distance_metric(row_1, row_2, covariance_matrix)

          if distancia < nearest_neighbours[0][0]:
            neighbour_label = int(training_label.at[index_2,'label'])
            nearest_neighbours[0] = [distancia, neighbour_label]
            nearest_neighbours.sort(reverse= True)

          #dos vizinhos dele, pegar o valor de label_data[index_2] que mais aparece
          
      neighbours_labels = [neighbours[1] for neighbours in nearest_neighbours]

      contador = Counter(neighbours_labels)
      prediction = contador.most_common(1)[0][0]

      predictions.append(prediction)
    return np.array(predictions)
  


## Questão 1

Recebendo os dados e fazendo o embaralhamento e normalização dos dados

In [3]:
data = pd.read_csv('kc2.csv')
data = data.sample(frac =1).reset_index(drop= True)

data_labels = data.iloc[:,-1:]
data_features = data.iloc[:, :-1]


In [4]:
Knn_1 = KNN(k = 1, distance_metric = euclidian_distance)
Knn_2 = KNN(k = 1, distance_metric = mahalanobis_distance)
Knn_3 = KNN(k= 5, distance_metric = euclidian_distance)
Knn_4 = KNN(k = 5, distance_metric= mahalanobis_distance)

Tree_1 = DecisionTreeClassifier(criterion = 'gini')
Tree_2 = DecisionTreeClassifier(criterion = 'entropy')

knn_k_1 = [Knn_1, Knn_2]
knn_k_5 = [Knn_3, Knn_4]
arvores = [Tree_1, Tree_2]


## KNN:

In [5]:
for model in knn_k_1:
    print('----='*10)
    print(f'\nMetricas para KNN com K = {model.k} e metrica usada {model.distance_metric_name}')
    k_fold(model,'KNN', data_features, data_labels)


----=----=----=----=----=----=----=----=----=----=

Metricas para KNN com K = 1 e metrica usada Distancia Euclidiana
Valor medio da acuracia: 0.7357954545454546 
Desvio padrao da acuracia: 0.11241531477457942

Valor medio da revocacao: 0.7508044733044733 
Desvio padrao da revocacao: 0.20069182159077828

Valor medio da precisao: 0.736916416916417 
Desvio padrao da precisao: 0.1121484791406097

Valor medio do f1_score: 0.729818376834126 
Desvio padrao do f1_score: 0.13042510289980785
----=----=----=----=----=----=----=----=----=----=

Metricas para KNN com K = 1 e metrica usada Distancia Mahalanobis
Valor medio da acuracia: 0.6914772727272728 
Desvio padrao da acuracia: 0.08075580794301376

Valor medio da revocacao: 0.6813167388167388 
Desvio padrao da revocacao: 0.15011626998982325

Valor medio da precisao: 0.7078496503496503 
Desvio padrao da precisao: 0.07356440817560757

Valor medio do f1_score: 0.6817065291687717 
Desvio padrao do f1_score: 0.0880185442871812


In [6]:
for model in knn_k_5:
    print('----='*10)
    print(f'\nMetricas para KNN com K = {model.k} e metrica usada {model.distance_metric_name}')
    k_fold(model,'KNN', data_features, data_labels)

----=----=----=----=----=----=----=----=----=----=

Metricas para KNN com K = 5 e metrica usada Distancia Euclidiana
Valor medio da acuracia: 0.7795454545454545 
Desvio padrao da acuracia: 0.07876237932347029

Valor medio da revocacao: 0.8020382395382395 
Desvio padrao da revocacao: 0.14307041054872488

Valor medio da precisao: 0.7826950826950827 
Desvio padrao da precisao: 0.07436918400303333

Valor medio do f1_score: 0.7833099360817761 
Desvio padrao do f1_score: 0.08044006503185785
----=----=----=----=----=----=----=----=----=----=

Metricas para KNN com K = 5 e metrica usada Distancia Mahalanobis
Valor medio da acuracia: 0.7306818181818182 
Desvio padrao da acuracia: 0.14517462411731363

Valor medio da revocacao: 0.6794011544011545 
Desvio padrao da revocacao: 0.21275151517970245

Valor medio da precisao: 0.7858372183372184 
Desvio padrao da precisao: 0.11124098941539753

Valor medio do f1_score: 0.7112539073600456 
Desvio padrao do f1_score: 0.14646485103374146


## Arvore de decisao para classificação

In [7]:
for model in arvores:
    print('----='*10)
    print(f'Métricas para arvore com o criterio de {model.criterion}:')
    k_fold(model= model, model_type= 'DT', features_data= data_features, label_data= data_labels)

----=----=----=----=----=----=----=----=----=----=
Métricas para arvore com o criterio de gini:
Valor medio da acuracia: 0.7431818181818182 
Desvio padrao da acuracia: 0.05754085864169167

Valor medio da revocacao: 0.7854942279942281 
Desvio padrao da revocacao: 0.09725232174545227

Valor medio da precisao: 0.7295404595404595 
Desvio padrao da precisao: 0.06093663790403986

Valor medio do f1_score: 0.7521545141018825 
Desvio padrao do f1_score: 0.049830168036713085
----=----=----=----=----=----=----=----=----=----=
Métricas para arvore com o criterio de entropy:
Valor medio da acuracia: 0.7113636363636364 
Desvio padrao da acuracia: 0.07190587281616535

Valor medio da revocacao: 0.7435028860028859 
Desvio padrao da revocacao: 0.11190395998244544

Valor medio da precisao: 0.7099206349206351 
Desvio padrao da precisao: 0.09017650869870954

Valor medio do f1_score: 0.718534677882504 
Desvio padrao do f1_score: 0.06040864519542899
